# <font style = "color:rgb(50,120,229)">How to stabilize landmark points in a video</font>

When you use Dlib’s Facial Landmark Detection on a video, you will notice they jiggle a bit. When the video is obtained under good and consistent lighting conditions, the landmarks tend to be more stable than when the lighting or imaging conditions are bad. 

The most important reason for this instability is that the landmarks are detected in every frame independently. There is nothing that ties the information in one frame to the information in the next. 

In this section, we will present a few strategies for stabilizing facial landmark points in videos. Depending on the algorithm we use for stabilizing the points, we will need up to four different pieces of information for stabilization

1. The location of a landmark in the current frame. 

2. The location of the same landmark in the previous frame(s). 

3. Pixels intensities in a small window in the current frame around the location of the landmark. 

4. Pixels intensities in a small window in the previous frame around the location of the landmark. 

**Note:** The methods we use are very general and can be applied to tracking points in general and not just facial landmarks. 

## <font style = "color:rgb(50,120,229)">Moving average : A simple solution</font>

The easiest solution to stabilize the points is to average the point locations over a small time window. This is called a moving average. 

For example, you can replace the current location of a landmark by the average location of the landmark over the last 5 frames. You could also use a weighted average, where the landmark location in the frame closest to the current frame is weighted more than the frames further away. 

The basic assumptions behind using moving average to stabilize landmark points are the following 

1. The motion of the points are slow compared to the frame rate of the video. 

2. The noise in the estimated landmark locations is zero mean. In other words, the process of averaging points over multiple frames will cancel out the noise. 

When the first assumption is violated, moving average produces points that lag the actual location of the landmarks. 

The moving averages algorithm is very easy to implement and therefore quite frequently used. 

## <font style = "color:rgb(50,120,229)">Kalman Filtering</font>

Computer Vision is not rocket science! But in Computer Vision, we do use technology that was initially used for tracking missiles and spacecrafts. One such technique is called Kalman Filtering invented by Rudolf E. Kálmán during the height of the cold war. 

In this approach, a prediction about the location of the landmark points ( or missiles for that matter ) is made based on the current state ( location + velocity ) of the landmark point. Kalman Filtering takes into account the measurement noise and the state is constantly updated based on quality of prediction on the current frame. This produces better results compared to moving averages, but requires more skill and understanding. However, like the moving average method, the pixel values are not used to estimate the location of landmark points. 

In this course, we will not cover Kalman Filtering because the method we describe next is more appropriate for tracking points on images. 

## <font style = "color:rgb(50,120,229)">Optical Flow</font>

In the methods described so far, only the location and velocity of the landmarks were used for tracking. However, because landmarks are points in an image, it makes sense to use a small patch around a landmarks in one frame to locate it in the next frame. 

The technique we describe next is called Optical Flow and it uses pixel information for making a prediction. 

### <font style = "color:rgb(8,133,37)">What is Optical Flow?</font>

Let us say, we are tracking a landmark point located at point $(x,y)$ in the image at frame $t$. In the next frame $(t+dt)$, the landmark moves to the point $(x+dx$ , $y+dy)$ where $dx$ , $dy$ are small displacements in the x and y directions. 

The optical flow at a point is simply the velocity vector of the point. It is given by 

$$
(u, v) = \left ( \frac{dx}{dt}, \frac{dy}{dt} \right )
$$  

When the optical flow is calculated over the entire image, it is called **dense optical flow** and when it is calculated over just a few points ( e.g. landmark points or feature points ), it is called **sparse optical flow.**

We want to calculate the optical flow at landmark locations, because once we have calculated the motion vector $(u,v)$, we are able to predict the location of landmark in the next frame based on the current frame by simply adding the the motion vector to the landmark location. 

We have additional material on Optical flow in the next section. You can go through them for more information on Optical Flow.

One of the methods of calculating Optical Flow is Lucas Kanade Algorithm. Again, we have additional material on Lucas Kanade algorithm in the next section. Here, we will just give a brief overview. You can go over Lucas Kanade algorithm in detail in the next section. Think of it as if we are using it to find the motion of particular landmark points in the next frame so that we can stabilize them.

## <font style = "color:rgb(50,120,229)">Lucas-Kanade Optical Flow</font>

This algorithm is used for finding optical flow at discrete points. In our case, we will use this algorithm to find the flow vectors at landmark points. 

Like any other optical flow algorithm, Lucas and Kanade had to make an assumption to solve the under-constrained optical flow equation. They made an assumption that in a small window centered at the feature point, the optical flow is the same for all pixels. So if we choose a 3x3 window around the feature point, there are 9 equations and 2 unknowns. And yes, this system of equations is easily solvable using simple linear algebra. 

Without going into the derivation of Lucas-Kanade optical flow, we are simply writing the final results in matrix form. 

$$
\begin{bmatrix} u \\ v  \end{bmatrix}  = \begin{bmatrix} \sum I^2_x & \sum I_x I_y \\   \sum I_y I_x  & \sum I^2_y \end{bmatrix}^{−1} \begin{bmatrix} \sum −I_x I_t \\ \sum −I_y I_t  \end{bmatrix}  
$$

Where the summation is carried over the small window and 

$$
I_x = \frac{\partial I}{\partial x}, I_y = \frac{\partial I}{\partial y} \text{ and } I_t = \frac{\partial I}{\partial t}
$$

## <font style = "color:rgb(50,120,229)">Handling Large Motion</font> 

The optical flow equation derived so far only holds for differential motion. Which means it will work when the motion is less than a pixel. Fortunately, there is a way to handle it using Image Pyramids. 

### <font style = "color:rgb(8,133,37)">What are Image Pyramids?</font>

An Image Pyramid is a multiscale representation of an image. The most common kind of Image Pyramid is called the **Gaussian Pyramid**.

<center> <a href="https://www.learnopencv.com/wp-content/uploads/2017/12/opcv4face-w3-m3-ImagePyramid.png"><img src = "https://www.learnopencv.com/wp-content/uploads/2017/12/opcv4face-w3-m3-ImagePyramid.png"/></a></center>

&nbsp;
The Gaussian Pyramid is produced by blurring the image using a Gaussian Kernel followed by downsizing the image by a factor of 2. Blurring is done to remove high frequency signal that can cause artifacts in a lower resolution image. 

For optical flow calculation, the motion vectors are calculated at the lowest level where the motion is indeed subpixel. The flow vectors are propagated up to the next higher level of the pyramid. A more accurate optical flow is then recalculated for this new level using the optical flow from the previous level as an initial guess. The resulting flow is propagated one level up. This process is repeated until the flow is calculated at the highest level.

**<font style="color:rgb(255,0,0)">Note:</font>** If the motion is extremely large, even the pyramids will not help. In such cases, we say that we have lost track of the point and it needs to be detected again. 

## <font style = "color:rgb(50,120,229)">Combining Detection and Tracking</font>

We have learned that we can predict the location of landmark points in the current frame given its location in the previous frame using optical flow. 

We also know the location of landmarks on the current frame calculated by the Facial Landmark Detector. 

Which of the two is more accurate?

Typically, when the motion is small and the appearance of the tracked landmark does not change between frames, tracking does a very good job. However, it is not uncommon for a tracker to lose track of the point completely because of large motion. 

On the other hand, the landmark detector usually provides a good rough location of the point. 

Therefore, when the motion of the face is small, we can choose the location predicted by optical flow. On the other hand, if the landmark locations predicted by the detector and the optical flow tracker are not close, we can assume the tracker has lost track and trust the landmark location predicted by the detector. 

However, if we make this binary decision to choose one prediction over the other, we will see the points jump around in a non-smooth way. So, we need to combine the two predictions in a more intelligent way. 

Here is the trick to combine the two estimates. 

Let, 

$p(t)$ = Position of a landmark in the current frame.

$p(t-1)$ = Position of the landmark in the previous frame.

$p_0(t)$ =  Position of the landmark predicted by optical flow in the current frame. 

Now, we need to combine $p(t)$ and $p_0(t)$ in some ratio. Let us call this ratio $\alpha$ , where $$ 0 \leq \alpha \leq 1 $$ We can find the stabilized point $p_s(t)$ using the following formula

$$  
p_s(t) = ( 1 − \alpha ) p(t) + \alpha p_o(t) 
$$

Stare at that equation for a bit and you will notice, when $\alpha=0.5$, the two predictions contribute equally to the final prediction. When $\alpha=0$, we fully trust the position given by the landmark detector - $p(t)$ and when $\alpha=1$ , fully trust the prediction given by optical flow - $p_0(t)$. 

As mentioned above, we also want this $\alpha$ to depend on how fast the point is moving. In other words, we want $\alpha$ to depend on the distance between the location of the point in the current frame ( i.e. $p(t)$) and the location of the point in the previous frame ( $p(t-1)$).

Let, 

$d$ = Distance between $p(t)$ and $p(t-1)$ = $||p(t)-p(t-1)||$ 

The distance $d$ can take any value between 0 and infinity ( well, theoretically speaking ). We need to map it between 0 and 1. To do this we define $\alpha$ to be the following 

$$
\alpha = e^{\frac{−d^2}{\sigma^2}}
$$

Looks exotic, but it is a simple way to map $d$ to a number between 0 and 1. What about $\sigma^2$? It is used to normalize $d$ because the distance $d$ is in pixels. For a high resolution image the $d$ will be higher compared to a lower resolution image for the same level of motion. So those numbers need to be scaled. In our implementation, $d$  is normalized by $\sigma$ which is proportional to the distance ( in pixels ) between the corners of the eyes. This is a better measure of scale than image resolution because we can have a small face in a high resolution image and vice versa. 

## <font style = "color:rgb(50,120,229)">Stabilizing landmarks using OpenCV</font>

As we know from previous sections, one way to implement stabilization requires optical flow calculation. Fortunately, OpenCV has a good implementation of Lukas Kanade optical flow described in this section. It can be invoked using **[`calcOpticalFlowPyrLK`](https://docs.opencv.org/4.1.0/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323)**.

```python

nextPts, status, err	=	cv.calcOpticalFlowPyrLK(	prevImg, nextImg, prevPts, nextPts[, status[, err[, winSize[, maxLevel[, criteria[, flags[, minEigThreshold]]]]]]]	)

```

Where,

- **`prevImg`** - first 8-bit input image or pyramid constructed by buildOpticalFlowPyramid.
- **`nextImg`** - second input image or pyramid of the same size and the same type as prevImg.
- **`prevPts`** - vector of 2D points for which the flow needs to be found; point coordinates must be single-precision floating-point numbers.
- **`nextPts`** - output vector of 2D points (with single-precision floating-point coordinates) containing the calculated new positions of input features in the second image; when OPTFLOW_USE_INITIAL_FLOW flag is passed, the vector must have the same size as in the input.
- **`status`** - output status vector (of unsigned chars); each element of the vector is set to 1 if the flow for the corresponding features has been found, otherwise, it is set to 0.
- **`err`** - output vector of errors; each element of the vector is set to an error for the corresponding feature, type of the error measure can be set in flags parameter; if the flow wasn't found then the error is not defined (use the status parameter to find such cases).
- **`winSize`** - size of the search window at each pyramid level.
maxLevel	0-based maximal pyramid level number; if set to 0, pyramids are not used (single level), if set to 1, two levels are used, and so on; if pyramids are passed to input then algorithm will use as many levels as pyramids have but no more than maxLevel.
- **`criteria`** - parameter, specifying the termination criteria of the iterative search algorithm (after the specified maximum number of iterations criteria.maxCount or when the search window moves by less than criteria.epsilon.
- **`flags`** - operation flags:
  - **`OPTFLOW_USE_INITIAL_FLOW`** - uses initial estimations, stored in nextPts; if the flag is not set, then prevPts is copied to nextPts and is considered the initial estimate.
  - **`OPTFLOW_LK_GET_MIN_EIGENVALS`** - use minimum eigen values as an error measure (see minEigThreshold description); if the flag is not set, then L1 distance between patches around the original and a moved point, divided by number of pixels in a window, is used as a error measure.
- **`minEigThreshold`** - the algorithm calculates the minimum eigen value of a 2x2 normal matrix of optical flow equations (this matrix is called a spatial gradient matrix), divided by number of pixels in a window; if this value is less than minEigThreshold, then a corresponding feature is filtered out and its flow is not processed, so it allows to remove bad points and get a performance boost.

As mentioned earlier, optical flow computation requires building Image pyramids for the current frame and the previous frame. **calcOpticalFlowPyrLK**does this calculation internally when you pass the previous frame and the current frame. When using optical flow in a video, the image pyramid for the same frame is built twice -- once while doing optical flow calculation for the current frame and the other for the next frame. This double calculation can be avoided by building and storing the image pyramid for every frame and passing it to **calcOpticalFlowPyrLK**.

The most common usage of **[`buildOpticalFlowPyramid`](https://docs.opencv.org/4.1.0/dc/d6b/group__video__track.html#ga86640c1c470f87b2660c096d2b22b2ce)** is shown below.

```python

retval, pyramid	=	cv.buildOpticalFlowPyramid(	img, winSize, maxLevel[, pyramid[, withDerivatives[, pyrBorder[, derivBorder[, tryReuseInputImage]]]]]	)

```

Where,


- **`img`** - 8-bit input image.
- **`pyramid`** - output pyramid.
- **`winSize`** - window size of optical flow algorithm. Must be not less than winSize argument of `calcOpticalFlowPyrLK`. It is needed to calculate required padding for pyramid levels.
- **`maxLevel`** - 0-based maximal pyramid level number.
- **`withDerivatives`** - set to precompute gradients for the every pyramid level. If pyramid is constructed without the gradients then calcOpticalFlowPyrLK will calculate them internally.
- **`pyrBorder`** - the border mode for pyramid layers.
- **`derivBorder`** - the border mode for gradients.
- **`tryReuseInputImage`** - put ROI of input image into the pyramid if possible. You can pass false to force data copying.

In [2]:
import cv2, dlib
import numpy as np
import math, sys
from dataPath import DATA_PATH
from dataPath import MODEL_PATH
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (6.0,6.0)
matplotlib.rcParams['image.cmap'] = 'gray'

In [6]:
PREDICTOR_PATH = MODEL_PATH + "shape_predictor_68_face_landmarks.dat"
RESIZE_HEIGHT = 480
NUM_FRAMES_FOR_FPS = 100
SKIP_FRAMES = 1

In [7]:
# Function to calculate the intereye distance.
def interEyeDistance(predict):
  leftEyeLeftCorner = (predict[36].x, predict[36].y)
  rightEyeRightCorner = (predict[45].x, predict[45].y)
  distance = cv2.norm(np.array(rightEyeRightCorner) - np.array(leftEyeLeftCorner))
  distance = int(distance)
  return distance

In [8]:
winName = "Stabilized facial landmark detector"

In [ ]:
videoFileName = ""

# Initializing video capture object.
cap = cv2.VideoCapture(videoFileName)

if(cap.isOpened()==False):
  print("Unable to load video")

### <font style="color:rgb(8,133,37)">Specifying the parameters of Lucas Kanade method </font>

In [ ]:
winSize = 101
maxLevel = 10
fps = 30.0
# Grab a frame
ret,imPrev = cap.read()

In [ ]:
# Convert to grayscale.
imGrayPrev = cv2.cvtColor(imPrev, cv2.COLOR_BGR2GRAY)

In [ ]:
# Finding the size of the image.
size = imPrev.shape[0:1]

In [ ]:
detector = dlib.get_frontal_face_detector()
landmarkDetector = dlib.shape_predictor(PREDICTOR_PATH)

### <font style="color:rgb(8,133,37)">Initializing the points </font>

In [ ]:
# Initializing the parameters
points=[]
pointsPrev=[]
pointsDetectedCur=[]
pointsDetectedPrev=[]

In [ ]:
eyeDistanceNotCalculated = True
eyeDistance = 0
isFirstFrame = True
# Initial value, actual value calculated after 100 frames
fps = 10
showStabilized = False
count =0

### <font style="color:rgb(8,133,37)">Use Detection + Tracking for stabilization </font>

As discussed above, we detect the landmarks in each frame and also use the Lucas Kanade method to track the points in the current frame w.r.t previous frame. Then we take a weighted average of the two measurements and that is the stabilized landmark point.

In [ ]:
while(True):
  if (count==0):
    t = cv2.getTickCount()

  # Grab a frame
  ret,im = cap.read()
  imDlib = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  # COnverting to grayscale
  imGray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  height = im.shape[0]
  IMAGE_RESIZE = float(height)/RESIZE_HEIGHT
  # Resize image for faster face detection
  imSmall = cv2.resize(im, None, fx=1.0/IMAGE_RESIZE, fy=1.0/IMAGE_RESIZE,interpolation = cv2.INTER_LINEAR)
  imSmallDlib = cv2.cvtColor(imSmall, cv2.COLOR_BGR2RGB)
  # Skipping the frames for faster processing
  if (count % SKIP_FRAMES == 0):
    faces = detector(imSmallDlib,0)

  # If no face was detected
  if len(faces)==0:
    print("No face detected")

  # If faces are detected, iterate through each image and detect landmark points
  else:
    for i in range(0,len(faces)):
      print("face detected")
      # Face detector was found over a smaller image.
      # So, we scale face rectangle to correct size.
      newRect = dlib.rectangle(int(faces[i].left() * IMAGE_RESIZE),
        int(faces[i].top() * IMAGE_RESIZE),
        int(faces[i].right() * IMAGE_RESIZE),
        int(faces[i].bottom() * IMAGE_RESIZE))
      
      # Detect landmarks in current frame
      landmarks = landmarkDetector(imDlib, newRect).parts()
      
      # Handling the first frame of video differently,for the first frame copy the current frame points
      
      if (isFirstFrame==True):
        pointsPrev=[]
        pointsDetectedPrev = []
        [pointsPrev.append((p.x, p.y)) for p in landmarks]
        [pointsDetectedPrev.append((p.x, p.y)) for p in landmarks]

      # If not the first frame, copy points from previous frame.
      else:
        pointsPrev=[]
        pointsDetectedPrev = []
        pointsPrev = points
        pointsDetectedPrev = pointsDetectedCur

      # pointsDetectedCur stores results returned by the facial landmark detector
      # points stores the stabilized landmark points
      points = []
      pointsDetectedCur = []
      [points.append((p.x, p.y)) for p in landmarks]
      [pointsDetectedCur.append((p.x, p.y)) for p in landmarks]

      # Convert to numpy float array
      pointsArr = np.array(points,np.float32)
      pointsPrevArr = np.array(pointsPrev,np.float32)

      # If eye distance is not calculated before
      if eyeDistanceNotCalculated:
        eyeDistance = interEyeDistance(landmarks)
        print(eyeDistance)
        eyeDistanceNotCalculated = False

      if eyeDistance > 100:
          dotRadius = 3
      else:
        dotRadius = 2

      print(eyeDistance)
      sigma = eyeDistance * eyeDistance / 400
      s = 2*int(eyeDistance/4)+1

      #  Set up optical flow params
      lk_params = dict(winSize  = (s, s), maxLevel = 5, criteria = (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, 20, 0.03))
      # Python Bug. Calculating pyramids and then calculating optical flow results in an error. So directly images are used.
      # ret, imGrayPyr= cv2.buildOpticalFlowPyramid(imGray, (winSize,winSize), maxLevel)

      pointsArr,status, err = cv2.calcOpticalFlowPyrLK(imGrayPrev,imGray,pointsPrevArr,pointsArr,**lk_params)
      

      # Converting to float
      pointsArrFloat = np.array(pointsArr,np.float32)

      # Converting back to list
      points = pointsArrFloat.tolist()

      # Final landmark points are a weighted average of
      # detected landmarks and tracked landmarks
      for k in range(0,len(landmarks)):
        d = cv2.norm(np.array(pointsDetectedPrev[k]) - np.array(pointsDetectedCur[k]))
        alpha = math.exp(-d*d/sigma)
        points[k] = (1 - alpha) * np.array(pointsDetectedCur[k]) + alpha * np.array(points[k])

      # Drawing over the stabilized landmark points
      if showStabilized is True:
        for p in points:
          cv2.circle(im,(int(p[0]),int(p[1])),dotRadius, (255,0,0),-1)
      else:
        for p in pointsDetectedCur:
          cv2.circle(im,(int(p[0]),int(p[1])),dotRadius, (0,0,255),-1)

      isFirstFrame = False
      count = count+1

      # Calculating the fps value
      if ( count == NUM_FRAMES_FOR_FPS):
        t = (cv2.getTickCount()-t)/cv2.getTickFrequency()
        fps = NUM_FRAMES_FOR_FPS/t
        count = 0
        isFirstFrame = True

      # Display the landmarks points
      cv2.putText(im, "{:.1f}-fps".format(fps), (50, size[0]-50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 255), 3,cv2.LINE_AA)
      cv2.imshow(winName, im)
      key = cv2.waitKey(25) & 0xFF

      # Use spacebar to toggle between Stabilized and Unstabilized version.
      if key==32:
        showStabilized = not showStabilized

      # Stop the program.
      if key==27:
        sys.exit()
      # Getting ready for next frame
      imPrev = im
      imGrayPrev = imGray

cv2.destroyAllwindows()
cap.release()

**<font style="color:rgb(255,0,0)">NOTE:</font>** Be careful with the code in Python. The **cv2.calcOpticalFlowPyrLK** returns a numpy array but for appending the points we are using a list, hence once we get the array we should convert it into a list. 

# <font style = "color:rgb(50,120,229)">References and Further readings</font>

1. [https://en.wikipedia.org/wiki/Moving_average](https://en.wikipedia.org/wiki/Moving_average)

2. [http://www.analog.com/media/en/technical-documentation/dsp-book/dsp_book_Ch15.pdf](http://www.analog.com/media/en/technical-documentation/dsp-book/dsp_book_Ch15.pdf)

3. [https://en.wikipedia.org/wiki/Lucas%E2%80%93Kanade_method](https://en.wikipedia.org/wiki/Lucas%E2%80%93Kanade_method)

4. [https://en.wikipedia.org/wiki/Kanade%E2%80%93Lucas%E2%80%93Tomasi_feature_tracker](https://en.wikipedia.org/wiki/Kanade%E2%80%93Lucas%E2%80%93Tomasi_feature_tracker)

5. [https://en.wikipedia.org/wiki/Kalman_filter](https://en.wikipedia.org/wiki/Kalman_filter)
